In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
#!pip install deap
import numpy as np
import random
#from deap import base
#from deap import creator
#from deap import tools
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler


import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

# Get the features and labels
data = load_breast_cancer()
x = data.data
y = data.target
print(data.feature_names)
# breast_cancer dataset

num_categories = len(np.unique(y))

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

# Standardize the features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Convert the labels to one-hot encoding
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

input_shape = (x_train.shape[1],)

# Define the model
model = Sequential([
        Dense(128, activation='relu', input_shape=input_shape),
        Dense(64, activation='relu'),
        Dense(num_categories, activation='softmax')
    ])

# Feedforwarding the weights
predictions = model.predict(x_train, verbose = 0)
predicted_labels = np.argmax(predictions, axis=1)

# Calculate the loss using categorical crossentropy
loss = tf.keras.losses.categorical_crossentropy(y_train_one_hot, predictions).numpy()

print("Loss:", np.mean(loss))

# Calculate accuracy
accuracy = accuracy_score(y_train, predicted_labels)
print("Accuracy:", accuracy)

# Get the weights of each layer
target_matrix = model.get_weights()
#print(target_matrix)


['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


D:\Programs\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Loss: 0.9741828
Accuracy: 0.21978021978021978


In [3]:
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt

# Constants
DIMENSIONS = 2
BOUND_LOW, BOUND_UP = -2, 2



durations = {}
# Function to generate a random neural network with given layer configurations
def generate_random_network(input_shape, num_categories):
    input_shape = (x_train.shape[1],)

    # Define the model
    model = Sequential([
            Dense(128, activation='relu', input_shape=input_shape),
            Dense(64, activation='relu'),
            Dense(num_categories, activation='softmax')
        ])
    return model

def initialize_population(POPULATION_SIZE, input_shape, num_categories):
    """
    Function to initialize the population of individuals (neural network weights).
    """
    population = []

    # Generate random weights for each individual in the population
    for _ in range(POPULATION_SIZE):
        # Generate a random neural network
        model = generate_random_network(input_shape, num_categories)

        # Get the weights of each layer
        individual_weights = model.get_weights()
        population.append(individual_weights)

        # Randomly initialize the weights
        for inv in population:
            for arr in range(len(inv)):
                if arr % 2 == 1:
                    for i in range(len(inv[arr])):
                        inv[arr][i] = random.uniform(-2, 2)  # Use np.random.uniform for continuous values

    return population

def mutate(i, population, F, model, x_train, y_train_one_hot):
    """
    Function to mutate an individual in the population.
    """
    mutated_matrix = []
    vectors_indexes = []
    vectors_indexes.append(i)
    # print(i)

    for _ in range(3):
        while len(vectors_indexes) < 4:
            random_index = random.randint(0, len(population) - 1)
            while random_index not in vectors_indexes:
                vectors_indexes.append(random_index)

    Target_matrix = population[vectors_indexes[0]]

    for j in range(len(Target_matrix)):
        mutation_value = population[vectors_indexes[3]][j] + F * (population[vectors_indexes[1]][j] - population[vectors_indexes[2]][j])
        mutated_matrix.append(mutation_value)
    clip_matrix = [np.clip(item, BOUND_LOW, BOUND_UP) if isinstance(item, np.ndarray) else np.clip(item, BOUND_LOW, BOUND_UP) for item in mutated_matrix]

    return clip_matrix

def crossover(mutated_matrix, Target_matrix, C, model, x_train, y_train_one_hot):
    """
    Function to perform crossover between mutated and trail matrices.
    """
    trail_matrix = mutated_matrix

    for arr in range(len(Target_matrix)):
        for elm in range(len(Target_matrix[arr])):
            random_num = np.random.rand()  # Generate a single random number
            if random_num < C:
                trail_matrix[arr][elm] = mutated_matrix[arr][elm]
            else:
                trail_matrix[arr][elm] = Target_matrix[arr][elm]

    return trail_matrix

def select_best(target_matrix, trail_matrix, model, x_train, y_train_one_hot):
    """
    Function to select the best between the target and trail matrices.
    """
    model.set_weights(target_matrix)
    target_predictions = model.predict(x_train, verbose=0)
    loss_target = tf.keras.losses.categorical_crossentropy(y_train_one_hot, target_predictions).numpy()
    target_predicted_labels = np.argmax(target_predictions, axis=1)

    model.set_weights(trail_matrix)
    trail_predictions = model.predict(x_train, verbose=0)
    loss_trail = tf.keras.losses.categorical_crossentropy(y_train_one_hot, trail_predictions).numpy()
    trail_predicted_labels = np.argmax(trail_predictions, axis=1)

    accuracy_target = accuracy_score(y_train, target_predicted_labels)
    accuracy_trail = accuracy_score(y_train, trail_predicted_labels)

    return target_matrix if np.mean(loss_target) < np.mean(loss_trail) else trail_matrix

def calculate_loss(individual, x_train, y_train_one_hot):
    """
    Function to calculate loss and accuracy for an individual.
    """
    # Set weights to the model
    model.set_weights(individual)

    # Predict using the model
    predictions = model.predict(x_train, verbose=0)

    # Calculate loss
    loss = tf.keras.losses.categorical_crossentropy(y_train_one_hot, predictions).numpy()
    predicted_labels = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(y_train, predicted_labels)

    return np.mean(loss), accuracy

def evolve_population(ps, num_generation, input_shape, num_categories, F, C, x_train, y_train_one_hot):
    """
    Function to evolve the population over a number of generations.
    """
    # Initialize the population
    population = initialize_population(ps, input_shape, num_categories)

    # Initialize the model
    model = generate_random_network(input_shape, num_categories)

    best_individual = None
    best_accuracy = 0

    history = {}
    durations = datetime.timedelta(0)

    for generation in range(1, num_generation + 1):
        start_time = datetime.datetime.now()

        print(f"\nGeneration no : {generation}")

        # Initialize new_population_parallel for each generation
        new_population_parallel = []

        # Your existing code for population mutation and other tasks
        # Inside the for loop for population mutation and other tasks
        for i in range(0, len(population)):
            mutated_matrix = mutate(i, population, F, model, x_train, y_train_one_hot)
            trail_matrix = crossover(mutated_matrix, population[i], C, model, x_train, y_train_one_hot)
            best_matrix = select_best(population[i], trail_matrix, model, x_train, y_train_one_hot)
            new_population_parallel.append(best_matrix)


        # Calculate loss and accuracy for each individual in the new population
        losses_and_accuracy = np.array([calculate_loss(individual, x_train, y_train_one_hot) for individual in new_population_parallel])
        # Unpack the tuple correctly
        losses = [loss for loss, _ in losses_and_accuracy]


        # Find the individual with the lowest loss
        if losses:
            lowest_loss = np.min(losses)
            accuracy = losses_and_accuracy[(losses_and_accuracy == np.min(losses)).any(axis=1)][0][1]
            idx = np.where((losses_and_accuracy == np.min(losses_and_accuracy[:, 0])))
            print(f"Lowest loss in generation {generation}: {lowest_loss} and Accuracy : {accuracy}")

        population = new_population_parallel
        history[generation] = [int(idx[0][0]),lowest_loss , accuracy]
        pop_loss = []
        pop_accuracy = []
        best_ind = new_population_parallel[int(idx[0][0])]

        end_time = datetime.datetime.now()

        duration = end_time - start_time
        print(f"Time : {duration}")
        durations +=  duration

        # Terminate the algorithm if the best loss is less than 0.15 and accuracy is greater than 0.9
        if lowest_loss < 0.15 and accuracy > 0.9:
            print("Terminating algorithm because best loss is less than 0.15 and accuracy is greater than 0.9")
            break

    print(f"Total generations run Time : {durations}")
    return history, durations, best_ind, population


## load the model 

In [4]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import datetime
from functools import partial

# Function to load saved networks
def load_networks(file_path):
    file_path ="model_train.h5"
    # Load the networks and return them
    return np.load(file_path, allow_pickle=True)


## GUI Code 


## Add Prediction option to the GUI 

In [5]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

def run_evolution(window, gen_entry, pop_size_entry, f_entry, c_entry, model_path_entry, predict_var, x_train, y_train_one_hot):
    num_generations = int(gen_entry.get()) if gen_entry.get() else 0
    ps = int(pop_size_entry.get()) if pop_size_entry.get() else 0
    F = float(f_entry.get()) if f_entry.get() else 0.0
    C = float(c_entry.get()) if c_entry.get() else 0.0
    model_path = model_path_entry.get()

    model = load_model(model_path)
    if predict_var.get() == 1:
        predictions = model.predict(x_train, verbose=0)
        predicted_labels = np.argmax(predictions, axis=1)

        loss = tf.keras.losses.categorical_crossentropy(y_train_one_hot, predictions).numpy()
        accuracy = accuracy_score(y_train, predicted_labels)
        
        print(f"Loss: {np.mean(loss)}, Accuracy: {accuracy}")
    else:
        input_shape = (x_train.shape[1],)
        num_categories = len(np.unique(y_train))
        history_breast_cancer, runTime_breast_cancer, evolv_weights, best_population = evolve_population(ps, num_generations, input_shape, num_categories, F, C, x_train, y_train_one_hot)

        labels = list(history_breast_cancer.keys())
        losses = [item[1] for item in history_breast_cancer.values()]
        accuracies = [item[2] for item in history_breast_cancer.values()]

        fig = plt.figure(figsize=(8, 6))

        plt.subplot(2, 1, 1)
        plt.plot(labels, losses, label='Loss')
        plt.xlabel("Generations")
        plt.ylabel("Loss")
        plt.title("Evolutionary Algorithm - Loss")

        plt.subplot(2, 1, 2)
        plt.plot(labels, accuracies, label='Accuracy')
        plt.xlabel("Generations")
        plt.ylabel("Accuracy")
        plt.title("Evolutionary Algorithm - Accuracy")

        plt.tight_layout()

        canvas = FigureCanvasTkAgg(fig, master=window)
        canvas.draw()
        canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)

def create_gui(x_train, y_train_one_hot):
    window = tk.Tk()
    window.title("Evolutionary Algorithm GUI")

    ttk.Label(window, text="Number of Generations:").pack()
    gen_entry = ttk.Entry(window)
    gen_entry.pack()

    ttk.Label(window, text="Population Size:").pack()
    pop_size_entry = ttk.Entry(window)
    pop_size_entry.pack()

    ttk.Label(window, text="F Value:").pack()
    f_entry = ttk.Entry(window)
    f_entry.pack()

    ttk.Label(window, text="C Value:").pack()
    c_entry = ttk.Entry(window)
    c_entry.pack()

    ttk.Label(window, text="Load Model Path:").pack()
    model_path_entry = ttk.Entry(window)
    model_path_entry.pack()

    load_model_button = ttk.Button(window, text="Load Model", command=lambda: load_model_path(model_path_entry))
    load_model_button.pack()

    predict_var = tk.IntVar()
    predict_var.set(1)
    ttk.Checkbutton(window, text="Predict", variable=predict_var, onvalue=1, offvalue=0).pack()
    ttk.Checkbutton(window, text="Evaluate", variable=predict_var, onvalue=0, offvalue=1, command=lambda: run_evolution(window, gen_entry, pop_size_entry, f_entry, c_entry, model_path_entry, predict_var, x_train, y_train_one_hot)).pack()

    run_button_text = "Run Evolution" if predict_var.get() == 1 else "Evaluate"
    run_button = ttk.Button(window, text=run_button_text, command=lambda: run_evolution(window, gen_entry, pop_size_entry, f_entry, c_entry, model_path_entry, predict_var, x_train, y_train_one_hot))
    run_button.pack()

    window.mainloop()

def load_model_path(entry):
    filename = filedialog.askopenfilename()
    entry.delete(0, tk.END)
    entry.insert(0, filename)

# Assuming x_train and y_train_one_hot are defined earlier in your code
# Example: x_train, y_train_one_hot = load_data()

# Create and run the GUI
create_gui(x_train, y_train_one_hot)


D:\Programs\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Generation no : 1
Lowest loss in generation 1: 0.5137633085250854 and Accuracy : 0.7164835164835165
Time : 0:00:02.671061

Generation no : 2
Lowest loss in generation 2: 0.5137633085250854 and Accuracy : 0.7164835164835165
Time : 0:00:03.079945

Generation no : 3
Lowest loss in generation 3: 0.3658366799354553 and Accuracy : 0.8505494505494505
Time : 0:00:02.691535

Generation no : 4
Lowest loss in generation 4: 0.3658366799354553 and Accuracy : 0.8505494505494505
Time : 0:00:03.061173

Generation no : 5
Lowest loss in generation 5: 0.3658366799354553 and Accuracy : 0.8505494505494505
Time : 0:00:02.789968

Generation no : 6
Lowest loss in generation 6: 0.3519609272480011 and Accuracy : 0.8571428571428571
Time : 0:00:03.302334

Generation no : 7
Lowest loss in generation 7: 0.3519609272480011 and Accuracy : 0.8571428571428571
Time : 0:00:02.966157

Generation no : 8
Lowest loss in generation 8: 0.3519609272480011 and Accuracy : 0.8571428571428571
Time : 0:00:03.591336

Generation no :

D:\Programs\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Generation no : 1
Lowest loss in generation 1: 0.42291197180747986 and Accuracy : 0.8505494505494505
Time : 0:01:03.353514

Generation no : 2
